In [1]:
import os

# 1. Khai báo tên model, pretrain_model và file cần làm việc

In [15]:
PRETRAIN_MODEL_NAME = 'faster_rcnn_resnet152_v1_640x640_coco17_tpu-8'
PRETRAIN_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py' #Tên file convert xml sang tfrecord
LABEL_MAP_NAME = 'label_map.pbtxt' #Tên file label
MAIN_FOLDER_PATH = os.getcwd()
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' # Tên của custom model
MAIN_FOLDER_PATH

'D:\\Code_school_nam3ki2\\KhoaHocDuLieu\\NhanDienMuBaoHiem'

# 2. Khai báo các đường dẫn thư mục làm việc

In [3]:
paths = {
    'WORKSPACE_PATH': os.path.join(MAIN_FOLDER_PATH, 'Tensorflow', 'workspace'), #Folder lảm việc, lưu dữ liệu khi train, valid và test
    'SCRIPTS_PATH': os.path.join(MAIN_FOLDER_PATH, 'Tensorflow','scripts'), #Folder chạy các file script liên quan đến convert ảnh
    'APIMODEL_PATH': os.path.join(MAIN_FOLDER_PATH, 'Tensorflow','models'), #Folder chứa tensorflow object detection api
    'ANNOTATION_PATH': os.path.join(MAIN_FOLDER_PATH, 'Tensorflow', 'workspace','annotations'), #Folder chứa các file config và file record
    'IMAGE_PATH': os.path.join(MAIN_FOLDER_PATH, 'Tensorflow', 'workspace','images'), #Folder chứa ảnh
    'MODEL_PATH': os.path.join(MAIN_FOLDER_PATH, 'Tensorflow', 'workspace','models'), #Folder chứa dữ liệu model khi làm việc(train, valid, test)
    'PRETRAINED_MODEL_PATH': os.path.join(MAIN_FOLDER_PATH,'Tensorflow', 'workspace','pre-trained-models'), #Folder chứa pretrain model
    'CHECKPOINT_PATH': os.path.join(MAIN_FOLDER_PATH,'Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), #Folder chứa các checkpoint
    'EXPORTED_MODEL_PATH': os.path.join(MAIN_FOLDER_PATH,'Tensorflow', 'workspace','exported-models', CUSTOM_MODEL_NAME), #Folder chứa model được lưu lại
    'PROTOC_PATH':os.path.join(MAIN_FOLDER_PATH,'Tensorflow','protoc') #Folder chứa protocol
}
paths

{'WORKSPACE_PATH': 'D:\\Code_school_nam3ki2\\KhoaHocDuLieu\\NhanDienMuBaoHiem\\Tensorflow\\workspace',
 'SCRIPTS_PATH': 'D:\\Code_school_nam3ki2\\KhoaHocDuLieu\\NhanDienMuBaoHiem\\Tensorflow\\scripts',
 'APIMODEL_PATH': 'D:\\Code_school_nam3ki2\\KhoaHocDuLieu\\NhanDienMuBaoHiem\\Tensorflow\\models',
 'ANNOTATION_PATH': 'D:\\Code_school_nam3ki2\\KhoaHocDuLieu\\NhanDienMuBaoHiem\\Tensorflow\\workspace\\annotations',
 'IMAGE_PATH': 'D:\\Code_school_nam3ki2\\KhoaHocDuLieu\\NhanDienMuBaoHiem\\Tensorflow\\workspace\\images',
 'MODEL_PATH': 'D:\\Code_school_nam3ki2\\KhoaHocDuLieu\\NhanDienMuBaoHiem\\Tensorflow\\workspace\\models',
 'PRETRAINED_MODEL_PATH': 'D:\\Code_school_nam3ki2\\KhoaHocDuLieu\\NhanDienMuBaoHiem\\Tensorflow\\workspace\\pre-trained-models',
 'CHECKPOINT_PATH': 'D:\\Code_school_nam3ki2\\KhoaHocDuLieu\\NhanDienMuBaoHiem\\Tensorflow\\workspace\\models\\my_ssd_mobnet',
 'EXPORTED_MODEL_PATH': 'D:\\Code_school_nam3ki2\\KhoaHocDuLieu\\NhanDienMuBaoHiem\\Tensorflow\\workspace\\expo

# 3. Khai báo các đường dẫn file làm việc

In [4]:
files = {
    'PIPELINE_CONFIG':os.path.join(MAIN_FOLDER_PATH,'Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'), #Path file pipelineconfig
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), #Path đến file convert từ xml sang tfrecord 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME) #Path label_config
}
files

{'PIPELINE_CONFIG': 'D:\\Code_school_nam3ki2\\KhoaHocDuLieu\\NhanDienMuBaoHiem\\Tensorflow\\workspace\\models\\my_ssd_mobnet\\pipeline.config',
 'TF_RECORD_SCRIPT': 'D:\\Code_school_nam3ki2\\KhoaHocDuLieu\\NhanDienMuBaoHiem\\Tensorflow\\scripts\\generate_tfrecord.py',
 'LABELMAP': 'D:\\Code_school_nam3ki2\\KhoaHocDuLieu\\NhanDienMuBaoHiem\\Tensorflow\\workspace\\annotations\\label_map.pbtxt'}

## Khởi tạo các đường dẫn thư mục và tải các thư viện cần thiết

In [5]:
for path in paths.values():
    if not os.path.exists(path):
          !mkdir -p {path}

In [31]:
!pip install wget

In [6]:
import wget

### Clone model tensorflow và object_detection

In [7]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {os.path.join(paths['APIMODEL_PATH'])}

### Cài đặt và giải nén protocol buffer, cài đặt và cấu hình môi trường protocol buffer cho việc sử dụng TensorFlow Object Detection API.

In [8]:
# Tải Protocol
if len(os.listdir(paths['PROTOC_PATH'])) == 0:
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    os.system(f"move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}")
    os.system(f"cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip")

In [57]:
# Thêm đường dẫn folder bin vào biến môi trường
os.environ['PATH'] += os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))

In [9]:
#Kiểm tra đường dẫn hợp lệ hay chưa
os.environ['PATH']

'C:\\Users\\minh3\\.conda\\envs\\py310;C:\\Users\\minh3\\.conda\\envs\\py310\\Library\\mingw-w64\\bin;C:\\Users\\minh3\\.conda\\envs\\py310\\Library\\usr\\bin;C:\\Users\\minh3\\.conda\\envs\\py310\\Library\\bin;C:\\Users\\minh3\\.conda\\envs\\py310\\Scripts;C:\\Users\\minh3\\.conda\\envs\\py310\\bin;C:\\ProgramData\\anaconda3\\condabin;C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v11.8\\bin;C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v11.8\\libnvvp;.;C:\\Program Files\\Python310\\Scripts;C:\\Program Files\\Python310;C:\\Program Files\\Common Files\\Oracle\\Java\\javapath;C:\\Windows\\system32;C:\\Windows;C:\\Windows\\System32\\Wbem;C:\\Windows\\System32\\WindowsPowerShell\\v1.0;C:\\Windows\\System32\\OpenSSH;C:\\Program Files (x86)\\NVIDIA Corporation\\PhysX\\Common;C:\\Program Files\\NVIDIA Corporation\\NVIDIA NvDLISR;C:\\WINDOWS\\system32;C:\\WINDOWS;C:\\WINDOWS\\System32\\Wbem;C:\\WINDOWS\\System32\\WindowsPowerShell\\v1.0;C:\\WINDOWS\\System32\\OpenSSH;C:\\Prog

In [ ]:
# Compile protobuf files and install all required libraries
os.system(f"cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install")

In [ ]:
os.system(f"cd Tensorflow/models/research/slim && pip install -e .")

### Kiểm tra tất cả thư viện đã tải về thành công

In [10]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

Running tests under Python 3.10.14: C:\Users\minh3\.conda\envs\py310\python.exe
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2024-05-20 17:23:28.421883: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 17:23:29.607656: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1668 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
C:\Users\minh3\.conda\envs\py310\lib\site-packages\object_detection\builders\model_builder.py:1112: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn(('Building experimental DeepMAC meta-arch.'
W0520

In [ ]:
# Tải lại tensorflow với version phù hợp(nếu Verify Installation bị lỗi)
!pip install tensorflow==2.13.0

In [36]:
import object_detection

### Cài đặt và giải nén pretrain_model

In [10]:
if len(os.listdir(paths['PRETRAINED_MODEL_PATH'])) == 0:
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

In [16]:
wget.download(PRETRAIN_MODEL_URL)
!move {PRETRAIN_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
!cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAIN_MODEL_NAME+'.tar.gz'}

100% [........................................................................] 20518283 / 20518283        1 file(s) moved.


x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/checkpoint
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0.index
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/pipeline.config
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/saved_model/
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/saved_model/saved_model.pb
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/saved_model/variables/
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/saved_model/variables/variables.index
